In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [220]:
cc_info = pd.read_csv('/Users/YaoJunyan/Documents/A collection of Data Science take-home challenge/Credit Card Transactions/credit_card/cc_info.csv')
transaction = pd.read_csv('/Users/YaoJunyan/Documents/A collection of Data Science take-home challenge/Credit Card Transactions/credit_card/transactions.csv')

In [221]:
cc_info.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [222]:
transaction.head()

,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719


In [223]:
#merge two datasets
df = transaction.merge(cc_info, how = 'left', on ='credit_card')

In [224]:
df.head()

,credit_card,date,transaction_dollar_amount,Long,Lat,city,state,zipcode,credit_card_limit
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,Houston,PA,15342,20000
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,Houston,PA,15342,20000
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,Houston,PA,15342,20000
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,Houston,PA,15342,20000
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,Houston,PA,15342,20000


### Question 1: Your boss wants to identify those users that in your dataset never went above the monthly credit card limit   alendar  onth . The goal of this is to automatically increase their limit. Can you send him the list of Ids?

In [225]:
df.date = pd.to_datetime(df.date)
df['transaction_year'] = df['date'].apply(lambda x: x.year)
df['transaction_month']= df['date'].apply(lambda x: x.month)
df['weekday'] = df['date'].apply(lambda x: x.dayofweek)
df['transaction_day'] = df['date'].apply(lambda x: x.day)
df['transaction_year_month'] = df['transaction_year'].map(str) + '-' + df['transaction_month'].map(str)

In [226]:
grouped_month = df.groupby(['credit_card','transaction_year_month','credit_card_limit'])['transaction_dollar_amount'].sum().reset_index()

In [227]:
grouped_month['exceed'] = (grouped_month['credit_card_limit'] < grouped_month['transaction_dollar_amount']).astype(int)

In [228]:
grouped_month.head()

,credit_card,transaction_year_month,credit_card_limit,transaction_dollar_amount,exceed
0,1003715054175576,2015-10,20000,8954.21,0
1,1003715054175576,2015-7,20000,162.56,0
2,1003715054175576,2015-8,20000,8441.51,0
3,1003715054175576,2015-9,20000,11281.56,0
4,1013870087888817,2015-10,15000,12090.34,0


In [229]:
grouped_credit_card = grouped_month.groupby('credit_card')['exceed'].sum().reset_index()

In [230]:
Increase_limit_list= grouped_credit_card[grouped_credit_card['exceed'] ==0]

In [231]:
Increase_limit_array= np.array(Increase_limit_list.credit_card)

In [232]:
len(Increase_limit_array)

862

### On the other hand, she wants you to implement an algorithm that as soon as a user goes above her monthly limit, it triggers an alert so that the user can be notified about that. We assume here that at the beginning of the new month, user total money spent gets reset to zero (i.e. she pays the card fully at the end of each month). Build a function that for each day, returns a list of users who went above their credit card monthly limit on that day.

In [233]:
def alert(data, year, month, day):
    data = data[(data['transaction_month'] == month)  & (data['transaction_day'] <= day)]
    grouped = data.groupby(['credit_card', 'credit_card_limit'])['transaction_dollar_amount'].sum().reset_index()
    return grouped[grouped['transaction_dollar_amount'] > grouped['credit_card_limit']]

### Finally, your boss is very concerned about frauds cause they are a huge cost for credit card companies. She wants you to implement an unsupervised algorithm that returns all transactions that seem unusual and are worth being investigated further.


My approach: Anomaly detection

* Simple statistical approach: Using percentile to check abnormal transactions
* ML approach: Kmeans



Some features we can consider
1. Transaction amount ratio: If the ratio is relatively high and not like this customer's regular transaction behavior, it might raise some attention
2. Transaction time: Some fraud activity might happen during night time
3. Transaction times: if the number of transactions per day exceeds this person's regular behavior, it might be fraudulant.
4. Transaction location: even though people might make transactions during travel time, but it's still good to check if this person's credit card transactions happened on the locations that far away from card holder's primary address

## Approach 1: K-means

In [234]:
## Feature Engineering

# Transaction to credit limit ratio

df['transaction_ratio'] = df['transaction_dollar_amount'] / df['credit_card_limit']

#Count of transaction per day
grouped_count = df.groupby(['credit_card', 'date'])['transaction_dollar_amount'].count().reset_index()
df = df.merge(grouped_count, how = 'left', on= ['credit_card', 'date'])
df =df.rename(columns={'transaction_dollar_amount_y': 'transaction_count', 'transaction_dollar_amount_x': 'transaction_dollar_amount'})

# transaction time
df['transaction_hour'] = df['date'].apply(lambda x: x.hour)
b = [0,4,8,12,16,20,24]
l = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
df['session'] = pd.cut(df['transaction_hour'], bins=b, labels=l, include_lowest=True)

In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294588 entries, 0 to 294587
Data columns (total 18 columns):
credit_card                  294588 non-null int64
date                         294588 non-null datetime64[ns]
transaction_dollar_amount    294588 non-null float64
Long                         294588 non-null float64
Lat                          294588 non-null float64
city                         294588 non-null object
state                        294588 non-null object
zipcode                      294588 non-null int64
credit_card_limit            294588 non-null int64
transaction_year             294588 non-null int64
transaction_month            294588 non-null int64
weekday                      294588 non-null int64
transaction_day              294588 non-null int64
transaction_year_month       294588 non-null object
transaction_ratio            294588 non-null float64
transaction_count            294588 non-null int64
transaction_hour             294588 non-null int64
se

In [244]:
df= pd.get_dummies(df, columns= ['session'])

In [246]:
X= df[['weekday', 'transaction_ratio', 'transaction_count', 'session_Late Night', 'session_Early Morning', 'session_Morning', 'session_Noon', 'session_Eve']]

In [247]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [250]:
df['kcluster'] = y_kmeans

In [254]:
len(df[df['kcluster']== 0])

202243

In [255]:
len(df[df['kcluster']== 1])

92345

In [256]:
# might be poor estimation

In [257]:
# Use another approach
def create_percentile(data):
    percentile = [25, 50, 75]
    d = np.percentile(data,percentile)
    return pd.Series(d, index= ['{}%'.format(p) for p in percentile])